## Data Processing & Engineering

### Reading in Data

In [1]:
# imports for processing

import pandas as pd
import numpy as np
from collections import Counter

In [2]:
scores = pd.read_csv('data/hearts_anon_raw.csv')
pre_eng_feats = scores.shape[1]
print(f"Current number of features: {pre_eng_feats}")
scores.head()

Current number of features: 5


,game_id,hand_id,player,received_cards_from,total_score
0,1,1,player_1,player_4,6
1,1,2,player_1,player_2,6
2,1,3,player_1,player_3,10
3,1,4,player_1,none,26
4,1,5,player_1,player_4,26


In [3]:
### Dictionary of the number of hands in each game

num_games = scores['game_id'].unique() # returns list of unique values of game_id column, which is the number of games played
player_list = scores.sort_values(by = "player", ascending = True)['player'].unique() # ordered list of player names
games_and_hands_dict = {} # dict with keys as game_id and values as total number of hands in each game

for game in num_games:

    hands_per_game = scores.query(f'game_id == {game}')['hand_id'].unique() # returns an array of ints from 1:max number of hands
    games_and_hands_dict[f'{game}'] = len(hands_per_game)

print (scores['received_cards_from'].unique()) # making sure there are only 4 players + "none" (and no spelling errors)
print (player_list) # making sure there are only 4 players (and no spelling errors)

games_and_hands_dict

['player_4' 'player_2' 'player_3' 'none' 'player_1']
['player_1' 'player_2' 'player_3' 'player_4']


{'1': 10,
 '2': 11,
 '3': 8,
 '4': 12,
 '5': 8,
 '6': 12,
 '7': 7,
 '8': 9,
 '9': 5,
 '10': 8,
 '11': 9,
 '12': 8,
 '13': 12,
 '14': 7}

### Computing 'points_per_hand' column

In [4]:
### points_per_hand -- DONE

for game in num_games:
    game_df = scores.query(f'game_id == {game}') # subsets original df per game_id, into game_df

    for player_name in player_list:
        player_df = game_df.query(f'player == "{player_name}"').sort_values(by = "hand_id", ascending = True) # subsets game_df per player, ascending by hand
        player_indices = player_df.index.tolist() # indices of player df

        for index in player_indices:
            if index == player_indices[0]:
                scores.loc[index, 'points_per_hand'] = scores.loc[index, 'total_score']
            else:
                scores.loc[index, 'points_per_hand'] = scores.loc[index, 'total_score'] - scores.loc[(index - 1), 'total_score']

scores.head()

,game_id,hand_id,player,received_cards_from,total_score,points_per_hand
0,1,1,player_1,player_4,6,6.0
1,1,2,player_1,player_2,6,0.0
2,1,3,player_1,player_3,10,4.0
3,1,4,player_1,none,26,16.0
4,1,5,player_1,player_4,26,0.0


### Computing 'percent_points_per_hand' column

In [5]:
### percent_points_per_hand -- DONE

for game in num_games:

    game_df = scores.query(f'game_id == {game}') # subsets original df per game_id, into game_df

    max_score_dict = {} # dictionary where keys = player names, values = each player's final score
    for player_name in player_list:
        game_per_player = game_df.query(f'player == "{player_name}"')
        game_per_player_indices = game_per_player.index.tolist()
        max_score_dict[f'{player_name}'] = game_per_player['total_score'].max()

        # print (game_per_player_indices)

        for index in game_per_player_indices:
            scores.loc[index, 'percent_points_per_hand'] = round((scores.loc[index, 'points_per_hand'] / max_score_dict[f'{player_name}'] * 100), 2)

    # print (max_score_dict)

scores.head()

,game_id,hand_id,player,received_cards_from,total_score,points_per_hand,percent_points_per_hand
0,1,1,player_1,player_4,6,6.0,6.12
1,1,2,player_1,player_2,6,0.0,0.00
2,1,3,player_1,player_3,10,4.0,4.08
3,1,4,player_1,none,26,16.0,16.33
4,1,5,player_1,player_4,26,0.0,0.00


### Computing 'queen_spades' column

In [6]:
### queen_spades -- DONE

queen_spades_list = []
for value in scores['points_per_hand']:
    if 13 <= value <= 25: # not a perfect solution but should get most of the cases accurate
        queen_spades_list.append(1)
    else:
        queen_spades_list.append(0)

queen_spades_list  
scores['queen_spades'] = queen_spades_list

scores.head()

,game_id,hand_id,player,received_cards_from,total_score,points_per_hand,percent_points_per_hand,queen_spades
0,1,1,player_1,player_4,6,6.0,6.12,0
1,1,2,player_1,player_2,6,0.0,0.00,0
2,1,3,player_1,player_3,10,4.0,4.08,0
3,1,4,player_1,none,26,16.0,16.33,1
4,1,5,player_1,player_4,26,0.0,0.00,0


### Computing 'moon_shooter' column

In [7]:
### moon_shooter -- DONE

scores['moon_shooter'] = "none" # sets all 'moon_shooter' values to "none" by default, since this is quicker than setting an if condition

for game in num_games:

    game_df = scores.query(f'game_id == {game}') # subsets original df per game_id, into game_df
    hands_per_game = game_df.query(f'game_id == {game}')['hand_id'].unique() # gets list of number of hands per game, 1 : x
    
    for hand in hands_per_game:

        hand_df = game_df.query(f'hand_id == {hand}') # subsets game_df per hand of game, into hand_df. Should be one row per player

        if hand_df['points_per_hand'].sum() == 78: # 78 = 26*3, which only happens when one player shoots the moon

            shooter_index = hand_df.query('points_per_hand == 0').index # returns index of player who shot the moon in that particular hand (1 int value)
            shooter_name = scores.loc[shooter_index]['player'].tolist()[0] # returns string of name player who shot the moon in that particular hand

            hand_index_list = hand_df.index.tolist() # gets list of indices of that hand
            
            scores.loc[hand_index_list, 'moon_shooter'] = shooter_name # changes 'moon_shooter' value to name of player who shot the moon at respective indices from index_list

scores.head()

,game_id,hand_id,player,received_cards_from,total_score,points_per_hand,percent_points_per_hand,queen_spades,moon_shooter
0,1,1,player_1,player_4,6,6.0,6.12,0,none
1,1,2,player_1,player_2,6,0.0,0.00,0,none
2,1,3,player_1,player_3,10,4.0,4.08,0,none
3,1,4,player_1,none,26,16.0,16.33,1,none
4,1,5,player_1,player_4,26,0.0,0.00,0,none


### Computing 'best_player_of_hand' column

In [8]:
### best_player_of_hand -- DONE

for game in num_games:

    game_df = scores.query(f'game_id == {game}') # subsets original df per game_id, into game_df
    hands_per_game = game_df.query(f'game_id == {game}')['hand_id'].unique() # gets list of number of hands per game, 1 : x
    
    for hand in hands_per_game:

        hand_df = game_df.query(f'hand_id == {hand}') # subsets game_df per hand of game, into hand_df. Should be one row per player
        
        best_score_of_hand = hand_df['points_per_hand'].min() # best score of that hand
        best_player_of_hand = hand_df.query(f'points_per_hand == {best_score_of_hand}').iloc[0]['player'] # name of best player of that hand
        
        hand_index_list = hand_df.index.tolist() # list of indices of that hand
        scores.loc[hand_index_list, 'best_player_of_hand'] = best_player_of_hand # assigning name of best player to all col vals for that hand

scores.head()

,game_id,hand_id,player,received_cards_from,total_score,points_per_hand,percent_points_per_hand,queen_spades,moon_shooter,best_player_of_hand
0,1,1,player_1,player_4,6,6.0,6.12,0,none,player_2
1,1,2,player_1,player_2,6,0.0,0.00,0,none,player_1
2,1,3,player_1,player_3,10,4.0,4.08,0,none,player_2
3,1,4,player_1,none,26,16.0,16.33,1,none,player_4
4,1,5,player_1,player_4,26,0.0,0.00,0,none,player_1


### Computing 'best_player_of_game' column

In [9]:
### best_player_of_game -- DONE

for game in num_games:
    game_df = scores.query(f'game_id == {game}') # subsets original df per game_id, into game_df
    best_player_of_game = game_df['best_player_of_hand'].mode().tolist()[0] # most common val (player name) in this col
    game_indices = game_df.index.tolist()
    
    scores.loc[game_indices, 'best_player_of_game'] = best_player_of_game # change vals of scores df based on this

scores.head()

,game_id,hand_id,player,received_cards_from,total_score,points_per_hand,percent_points_per_hand,queen_spades,moon_shooter,best_player_of_hand,best_player_of_game
0,1,1,player_1,player_4,6,6.0,6.12,0,none,player_2,player_1
1,1,2,player_1,player_2,6,0.0,0.00,0,none,player_1,player_1
2,1,3,player_1,player_3,10,4.0,4.08,0,none,player_2,player_1
3,1,4,player_1,none,26,16.0,16.33,1,none,player_4,player_1
4,1,5,player_1,player_4,26,0.0,0.00,0,none,player_1,player_1


### Computing 'game_winner' column

In [10]:
### game_winner -- DONE

for game in num_games:

    game_df = scores.query(f'game_id == {game}') # subsets original df per game_id, into game_df
    game_indices = game_df.index.tolist()
    
    last_hand_value = len(game_df.query(f'game_id == {game}')['hand_id'].unique()) # gets last hand of each game
    last_hand_df = game_df.query(f'hand_id == {last_hand_value}') # df of just the last hand
    game_winner_name = last_hand_df.sort_values(by = "total_score", ascending = True)["player"].tolist()[0]

    scores.loc[game_indices, 'game_winner'] = game_winner_name # assigning name of game winner to game_winner col vals for that game

scores

,game_id,hand_id,player,received_cards_from,total_score,points_per_hand,percent_points_per_hand,queen_spades,moon_shooter,best_player_of_hand,best_player_of_game,game_winner
0,1,1,player_1,player_4,6,6.0,6.12,0,none,player_2,player_1,player_2
1,1,2,player_1,player_2,6,0.0,0.00,0,none,player_1,player_1,player_2
2,1,3,player_1,player_3,10,4.0,4.08,0,none,player_2,player_1,player_2
3,1,4,player_1,none,26,16.0,16.33,1,none,player_4,player_1,player_2
4,1,5,player_1,player_4,26,0.0,0.00,0,none,player_1,player_1,player_2
...,...,...,...,...,...,...,...,...,...,...,...,...
499,14,3,player_4,player_2,55,25.0,25.00,1,none,player_1,player_1,player_1
500,14,4,player_4,none,55,0.0,0.00,0,none,player_2,player_1,player_1
501,14,5,player_4,player_3,69,14.0,14.00,1,none,player_1,player_1,player_1
502,14,6,player_4,player_1,94,25.0,25.00,1,none,player_1,player_1,player_1


### Computing 'hand_position' column
- The relative position of each player in each hand of the game

In [11]:
for game in num_games:
    game_df = scores.query(f'game_id == {game}') # subsets original df per game_id, into game_df
    game_indices = game_df.index.tolist()

    num_hands = game_df['hand_id'].unique().tolist()

    for hand in num_hands:
        hand_df = game_df.query(f'hand_id == {hand}').sort_values(by = 'total_score', ascending = True)
        hand_indices = hand_df.index.tolist()

        for val in hand_indices:
            scores.loc[val, 'hand_position'] = hand_indices.index(val) + 1
# scores

In [12]:
scores.query("game_id == 1 & hand_id == 1").sort_values(by = 'total_score', ascending = True)
# scores.head(40)

,game_id,hand_id,player,received_cards_from,total_score,points_per_hand,percent_points_per_hand,queen_spades,moon_shooter,best_player_of_hand,best_player_of_game,game_winner,hand_position
30,1,1,player_2,player_1,1,1.0,2.78,0,none,player_2,player_1,player_2,1.0
10,1,1,player_3,player_2,5,5.0,4.59,0,none,player_2,player_1,player_2,2.0
0,1,1,player_1,player_4,6,6.0,6.12,0,none,player_2,player_1,player_2,3.0
20,1,1,player_4,player_3,14,14.0,20.29,1,none,player_2,player_1,player_2,4.0


In [13]:
# ### This checks to make sure there's one player in each position (1 through 4) in each hand

# for game in num_games:
#     game_df = scores.query(f'game_id == {game}') # subsets original df per game_id, into game_df
#     game_indices = game_df.index.tolist()

#     num_hands = game_df['hand_id'].unique().tolist()
#     # print ("\n")
#     for hand in num_hands:
#         hand_df = game_df.query(f'hand_id == {hand}').sort_values(by = 'total_score', ascending = False)
#         ranks = hand_df['hand_position'].unique().tolist()
#         print (f"Game: {game} -- Hand: {hand} -- Ranks: {str(ranks)}")
#     print ("\n")

### Computing 'swing_down', 'swing_up', 'no_swing' columns

In [14]:
scores['swing_down'] = 0 # initial a minimum count for the number of times someone down in position (was surpassed) per game 
scores['swing_up'] = 0 # initial a minimum count for the number of times someone moved up in position (surpassed someone) per game
scores['no_swing'] = 0 # initial a minimum count for the number of times someone moved up in position (surpassed someone) per game

for game in num_games:

    for player in player_list:

        game_per_player_df = scores.query(f'game_id == {game} & player == "{player}"') # subsets original df per game_id and player_id, into game_df
        game_per_player_df
        indices = game_per_player_df.index.tolist()

        for ind in indices:
            if game_per_player_df.loc[ind, 'hand_id'] == 1: # makes sure everyone starts from a neutral position in hand one, starting at 0 for all 3 cols. Position is decided after first hand
                scores.loc[ind, 'swings'] = "start"
            elif game_per_player_df.loc[ind, 'hand_id'] != (game_per_player_df['hand_id'].max() or indices[0]):
                hand_pos1 = game_per_player_df.loc[ind, 'hand_position']
                hand_pos2 = game_per_player_df.loc[indices[indices.index(ind) + 1], 'hand_position']

                if hand_pos1 > hand_pos2:
                    # scores.loc[ind, 'swing_up'] = scores.loc[indices[indices.index(ind) - 1, 'swing_up']] + 1
                    scores.loc[ind, 'swing_up'] += 1
                    scores.loc[ind, 'swings'] = "up"
                elif hand_pos1 < hand_pos2:
                    scores.loc[ind, 'swing_down'] += 1
                    scores.loc[ind, 'swings'] = "down"
                elif hand_pos1 == hand_pos2:
                    scores.loc[ind, 'no_swing'] += 1
                    scores.loc[ind, 'swings'] = "same"

            else:
                hand_pos2 = game_per_player_df.loc[ind, 'hand_position']

                if hand_pos1 > hand_pos2:
                    scores.loc[ind, 'swing_up'] += 1
                    scores.loc[ind, 'swings'] = "up"
                elif hand_pos1 < hand_pos2:
                    scores.loc[ind, 'swing_down'] += 1
                    scores.loc[ind, 'swings'] = "down"
                elif hand_pos1 == hand_pos2:
                    scores.loc[ind, 'no_swing'] += 1
                    scores.loc[ind, 'swings'] = "same"

# scores

In [15]:
scores.query("game_id == 1 & player == 'player_1'")

,game_id,hand_id,player,received_cards_from,total_score,points_per_hand,percent_points_per_hand,queen_spades,moon_shooter,best_player_of_hand,best_player_of_game,game_winner,hand_position,swing_down,swing_up,no_swing,swings
0,1,1,player_1,player_4,6,6.0,6.12,0,none,player_2,player_1,player_2,3.0,0,0,0,start
1,1,2,player_1,player_2,6,0.0,0.00,0,none,player_1,player_1,player_2,2.0,0,0,1,same
2,1,3,player_1,player_3,10,4.0,4.08,0,none,player_2,player_1,player_2,2.0,0,0,1,same
3,1,4,player_1,none,26,16.0,16.33,1,none,player_4,player_1,player_2,2.0,0,0,1,same
4,1,5,player_1,player_4,26,0.0,0.00,0,none,player_1,player_1,player_2,2.0,0,0,1,same
5,1,6,player_1,player_2,31,5.0,5.10,0,none,player_4,player_1,player_2,2.0,0,0,1,same
6,1,7,player_1,player_3,57,26.0,26.53,0,player_3,player_3,player_1,player_2,2.0,1,0,0,down
7,1,8,player_1,none,81,24.0,24.49,1,none,player_3,player_1,player_2,3.0,1,0,0,down
8,1,9,player_1,player_4,98,17.0,17.35,1,none,player_3,player_1,player_2,4.0,0,1,0,up
9,1,10,player_1,player_2,98,0.0,0.00,0,none,player_1,player_1,player_2,3.0,0,1,0,up


### Final scores df & Writing to New CSV

In [16]:
# scores.head(60) # can only show first 60 rows in output
post_eng_feats = scores.shape[1]
print(f"Number of features added: {post_eng_feats - pre_eng_feats}")
scores.query("hand_id != 1").head(40)
scores.head(10)

Number of features added: 12


,game_id,hand_id,player,received_cards_from,total_score,points_per_hand,percent_points_per_hand,queen_spades,moon_shooter,best_player_of_hand,best_player_of_game,game_winner,hand_position,swing_down,swing_up,no_swing,swings
0,1,1,player_1,player_4,6,6.0,6.12,0,none,player_2,player_1,player_2,3.0,0,0,0,start
1,1,2,player_1,player_2,6,0.0,0.00,0,none,player_1,player_1,player_2,2.0,0,0,1,same
2,1,3,player_1,player_3,10,4.0,4.08,0,none,player_2,player_1,player_2,2.0,0,0,1,same
3,1,4,player_1,none,26,16.0,16.33,1,none,player_4,player_1,player_2,2.0,0,0,1,same
4,1,5,player_1,player_4,26,0.0,0.00,0,none,player_1,player_1,player_2,2.0,0,0,1,same
5,1,6,player_1,player_2,31,5.0,5.10,0,none,player_4,player_1,player_2,2.0,0,0,1,same
6,1,7,player_1,player_3,57,26.0,26.53,0,player_3,player_3,player_1,player_2,2.0,1,0,0,down
7,1,8,player_1,none,81,24.0,24.49,1,none,player_3,player_1,player_2,3.0,1,0,0,down
8,1,9,player_1,player_4,98,17.0,17.35,1,none,player_3,player_1,player_2,4.0,0,1,0,up
9,1,10,player_1,player_2,98,0.0,0.00,0,none,player_1,player_1,player_2,3.0,0,1,0,up


In [17]:
### Writing processed df to new csv

scores.to_csv('data/hearts_anon_processed.csv')